# bm25 llamaindex 支持中文

## 准备工作

In [2]:
%%time
%%capture

%pip install rank_bm25
%pip install nltk jieba

!pip install llama-index-retrievers-bm25==0.1.3
!pip install rank_bm25
!pip install jieba

CPU times: user 43.3 ms, sys: 7.09 ms, total: 50.3 ms
Wall time: 16.8 s


In [3]:
%%time

# 下载停用词

# 设置 HTTP 代理环境变量
# https://github.com/nltk/nltk_data/issues/154#issuecomment-2144880495
http_proxy="http://192.168.0.134:7890"

import nltk
nltk.set_proxy(f'{http_proxy}')
nltk.download('stopwords')

CPU times: user 655 ms, sys: 191 ms, total: 846 ms
Wall time: 1.05 s


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
%%time
import jieba
from typing import List
from nltk.corpus import stopwords

def chinese_tokenizer(text: str) -> List[str]:
    # Use jieba to segment Chinese text
    return list(jieba.cut(text))
    # return list(jieba.lcut(text))

CPU times: user 65.2 ms, sys: 20.4 ms, total: 85.7 ms
Wall time: 96.8 ms


In [5]:
chinese_tokenizer("床前明月光")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.421 seconds.
Prefix dict has been built successfully.


['床前', '明月光']

## 简单示例 - 唐诗

### 创建文档

In [6]:
%%time

from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.response.notebook_utils import display_source_node

documents = [Document(text="床前明月光"),
             Document(text="疑是地上霜"),
             Document(text="举头望明月"),
             Document(text="低头思故乡")]

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

nodes

CPU times: user 2.48 s, sys: 194 ms, total: 2.67 s
Wall time: 2.77 s


[TextNode(id_='dedaf6f4-fe19-4545-86e0-8d307ea3d537', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='655161fd-8613-4829-82a5-26626eb1c7e0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='ac0f389f8d076d84f369d702eaf943bb5a090d8aa88d9db41a0f79cbc098ea86')}, text='床前明月光', start_char_idx=0, end_char_idx=5, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 TextNode(id_='dbc4c562-cac7-4e86-a77c-f9a49702bf51', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='63ef91c7-b779-4c9b-b7a8-13d4c24e0dbc', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='761a48a5e3191f70fdb9c7799ef3c6168a3cc0db449cb4a7b40b0b450fd66fd9')}, text='疑是地上霜', start_char_idx=0, end_char_idx=5, text_template='{metadata_str}

### 使用 retriver

In [7]:
%%time

retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=1,
    tokenizer=chinese_tokenizer,
)

CPU times: user 169 µs, sys: 20 µs, total: 189 µs
Wall time: 192 µs


In [8]:
retriever.retrieve("故乡")

[NodeWithScore(node=TextNode(id_='7b685b22-b73c-48be-a196-1c79ceeb6e6d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cd1cf2b1-5048-4a88-bdd5-12a031b5845e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='d5b6840ed7517b443f5a2bf10b5aa511799e6715c432857123f40212514233b2')}, text='低头思故乡', start_char_idx=0, end_char_idx=5, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.777337486593765)]

In [9]:
retriever.retrieve("举头")

[NodeWithScore(node=TextNode(id_='357302a9-19aa-4a25-8603-f07726a96c7d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e6af8627-7dca-44ec-b395-81df59ba2f4f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='42bf7864167ad4408fb5929d538790398e1be8b5c36dffb328dc493f94cd0f64')}, text='举头望明月', start_char_idx=0, end_char_idx=5, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.9310965498760481)]

## 复杂示例 - 新闻摘要

### 创建文档

In [10]:
%%time

documents = [Document(text="欧盟委员会首席发言人埃里克·马梅宣布，由于匈牙利担任欧盟理事会轮值主席国以来的事态发展，欧盟委员会主席冯德莱恩决定只派高级公务员代表参加理事会非正式会议，而不是派遣高级官员。匈牙利方面对此表示质疑，认为欧盟委员会不能挑选它想要合作的机构和成员国。"),
             Document(text="文章由专栏作者青岚撰写，主要探讨了荷兰光刻机巨头ASML公司在四十年的芯片产业发展中的变迁，以及它如何从最初的市场挑战者变成了当前全球半导体产业政治博弈中的焦点。"),
             Document(text="文章由华阳海洋研究中心理事长吴士存撰写，讨论了近期中菲在南海仁爱礁的对峙问题，并提出了中国应对该问题的建议。"),
             Document(text="报道主要关注了孟加拉国总理哈西娜在提斯塔河项目上的立场，以及孟加拉国和印度在水资源分配问题上的长期争端。哈西娜倾向于让印度负责项目，以确保孟加拉国的水资源需求得到满足。")]

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

CPU times: user 2.55 ms, sys: 311 µs, total: 2.86 ms
Wall time: 2.12 ms


### 使用 retriver

In [11]:
%%time

retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=1,
    tokenizer=chinese_tokenizer,
)

CPU times: user 1.03 ms, sys: 124 µs, total: 1.15 ms
Wall time: 1.16 ms


In [12]:
%%time

retriever.retrieve("欧盟匈牙利轮值主席国后发生了啥")

CPU times: user 1.38 ms, sys: 0 ns, total: 1.38 ms
Wall time: 1.1 ms


[NodeWithScore(node=TextNode(id_='66ecca59-522d-49f2-be65-1574e2dc334b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a0c2d4aa-479d-40e5-83ed-da976755d019', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='da0f80d48d2c7374c50a4b3ef22dce19c723c2e43db9a6c6110637d697f5466a')}, text='欧盟委员会首席发言人埃里克·马梅宣布，由于匈牙利担任欧盟理事会轮值主席国以来的事态发展，欧盟委员会主席冯德莱恩决定只派高级公务员代表参加理事会非正式会议，而不是派遣高级官员。匈牙利方面对此表示质疑，认为欧盟委员会不能挑选它想要合作的机构和成员国。', start_char_idx=0, end_char_idx=123, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=3.375648286021731)]

In [13]:
%%time

retriever.retrieve("荷兰asml公司发展有哪些故事")

CPU times: user 679 µs, sys: 83 µs, total: 762 µs
Wall time: 757 µs


[NodeWithScore(node=TextNode(id_='0a02f38b-89dd-457c-86f0-a9963441a151', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2a8d6d4e-cbae-4f22-94ae-58feeb11d325', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='678927f4d186eb02e8f1ca9fd066c9c9fa333e2a8d25bf50904d2de9f094c987')}, text='文章由专栏作者青岚撰写，主要探讨了荷兰光刻机巨头ASML公司在四十年的芯片产业发展中的变迁，以及它如何从最初的市场挑战者变成了当前全球半导体产业政治博弈中的焦点。', start_char_idx=0, end_char_idx=81, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1.6903462469691775)]

In [14]:
%%time

retriever.retrieve("仁爱礁下一步会发生什么事情")

CPU times: user 1.36 ms, sys: 0 ns, total: 1.36 ms
Wall time: 1.23 ms


[NodeWithScore(node=TextNode(id_='e71163d6-887a-4aa2-99b5-79064ea80a45', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a7239a07-35f0-484e-97d1-ee7b5129f387', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='e7d871f0cb2a5b3fca85ef67599dec9c170018551a96b3b29db2589bdc49c7df')}, text='文章由华阳海洋研究中心理事长吴士存撰写，讨论了近期中菲在南海仁爱礁的对峙问题，并提出了中国应对该问题的建议。', start_char_idx=0, end_char_idx=54, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1.9438169434067216)]